In [1]:
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

/home/james/priv_pred_ensemble/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("GPT2")
f_0 = GPT2LMHeadModel.from_pretrained('GPT2', pad_token_id=t)

NameError: name 'GPT2' is not defined